In [ ]:
import sys
import os
sys.path.append("../Calculator/")
from PDF import PDF
from OLEDSimPlot import *
from multiprocessing import Pool

In [ ]:
materials=[
Metal(150*10**-9, -4.8*e, name="ITO"),
 Metal(20*10**-9, -5.0*e, name="PEDOT:PSS"),
 Organic(70*10**-9, -2.3*e, -5.2*e, name="poly-TPD", sigma=0.1*e, dim=0.5*10**-9, height=3),
 DopedOrganic(10*10**-9, -2.9*e, -6*e, -3*e, -5.6*e, name="CBP", name2="Ir(ppy)\\textsubscript{3}",outsourceDesc=-1.0, sigma=0.1*e, dim=0.5*10**-9), #sigmaaldrich
 Organic(10*10**-9, -3.0*e, -6.5*e, name="OXD-7" , sigma=0.1*e, dim=0.5*10**-9, polarity=0, height=3/4),
 Organic(65*10**-9, -3.1*e, -5.8*e, name="Alq$_\\text{3}$", sigma=0.1*e, dim=0.5*10**-9, polarity=0),
 Metal(10*10**-9, -2.83*e, name="Ca"),
 Metal(100*10**-9, -4.2*e, name="Al")]


In [ ]:
#obligatory
title=""
name=""
desiredPlots=[
    {
        "xCol":1,
        "yCol":2
    }
    ]
scaled=False

In [ ]:
#optional
figscale=0.8
present={"customFontsize":[10,10,6,8,8],
        "fig_width_pt":342.29536*figscale,
        "scaleX":(342.29536*figscale)/448.13095,
        "fixedFigWidth":True,
        "HWratio":3/4,
        "titleBool":False,
        "filename":"presentation"}
inputParameters={
                    "customFontsize":[12,12,12,10,10],
                    "title":title,
                    "titleBool":False,
                }
inputParameters.update(present)
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.47,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })

In [ ]:
def initPlot(xCol=1, showColTup=(2,0), xCol2=0, customInputParameters=None):
    if customInputParameters is not None:
        inputParameters.update(customInputParameters)
        inputParametersForScaled.update(customInputParameters)
    scPlot=None
    plot=ReflectoPlot(name, 
                      materials,
                      xCol=xCol,
                      xCol2=xCol2,
                      showColTup=showColTup,           
                      **inputParameters)
    if scaled:
        scPlot=ReflectoPlot(name,
                            materials,
                            xCol=xCol,
                            xCol2=xCol2,
                            showColTup=showColTup,
                            **inputParametersForScaled)
    if scPlot is None:
        return [plot]
    return [plot,scPlot]

In [ ]:
def buildPlotList(desiredPlot):
    try:
        yCol2=desiredPlot["yCol2"]
    except KeyError:
        yCol2=0
    try:
        xCol2=desiredPlot["xCol2"]
    except KeyError:
        xCol2=0    
    try:
        cusPara=desiredPlot["custom"]
    except KeyError:
        cusPara=None
    if desiredPlot["yCol"]==0 or desiredPlot["xCol"]==0:
        raise
    return initPlot(xCol=desiredPlot["xCol"], showColTup=(desiredPlot["yCol"],yCol2), xCol2=xCol2, customInputParameters=cusPara)

In [ ]:
def processPlotPair(plotpair):
    return [plot.doPlot() for plot in plotpair]

In [ ]:
#multithreaded
pool = Pool(os.cpu_count())
plotList=pool.map(buildPlotList,desiredPlots)
multiOutput=pool.map(processPlotPair,plotList)
pool.close()

In [ ]:
plots=[[plotOutput[0] for plotOutput in plotPair] for plotPair in multiOutput]
files=[[plotOutput[1] for plotOutput in plotPair] for plotPair in multiOutput]

In [ ]:
PDF(files[0][0], size=(700,700*plots[0][0].HWratio)) 